<a href="https://colab.research.google.com/github/satishchaudhary382/algorithmic-trading-with-python/blob/master/Master%20S.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [157]:
!pip install pandas_ta

In [2]:
import pandas_datareader as pdr
import pandas_ta as ta

df = pdr.get_data_yahoo("ebay","1-1-2020")



In [224]:
#@title TRIMA band
def trima(df):
  df['trima'] = ta.trima(df['Close'],21)
  df['range'] = ta.atr(df.High, df.Low, df.Close, 14) * 4
  df['H'] = ta.wma((df['trima'] + df['range']),21)
  df['L'] = ta.wma((df['trima'] - df['range']),21)
  df['SD_UP'] = df['H'].rolling(20).std()
  # df['SD_DOWN'] = ((df['trima'] - df['range'])).rolling(20).std()
  df['HH'] = ta.wma((df['trima'] + df['range']),21)+(ta.atr(df.High, df.Low, df.Close,21))*1
  df['LL'] = ta.wma((df['trima'] - df['range']),21)-(ta.atr(df.High, df.Low, df.Close,21))*1
  return df[['HH','H','LL','L']]

In [39]:
#@title
import pandas as pd
import matplotlib.pyplot as plt

df = pdr.get_data_yahoo("NVDA", "9-1-2020")
aLength = 2
yLength = 5
zLength = 25
obLevel = 70
osLevel = 30
df['hc'] = df['Close'].rolling(aLength).max()
df['lc'] = df['Close'].rolling(aLength).min()
df['top'] = ta.ema(ta.ema((df['Close']-df['lc']), yLength),zLength)
df['bot'] = ta.ema(ta.ema((df['hc'] - df['Close']), yLength),zLength)
df['rs'] = df.apply(lambda x: x['top']/x['bot'] if x['bot']!=0 else 0, axis = 1)
df['rsi'] = df.apply(lambda x: 100 if x['bot']==0 else 0 if x['top']==0 else 100-(100/(1+x['rs'])),axis =1)
df['rsi_ema'] = ta.ema(df['rsi'], 21)


In [ ]:
#@title
# df[['rsi_ema','rsi']].plot(figsize = (20,10))
fig, (ax1,ax2) = plt.subplots(nrows=2, sharex=True, subplot_kw=dict(frameon=False), figsize = (20,10)) # frameon=False removes frames

plt.subplots_adjust(hspace=.05)
ax1.grid()
ax2.grid()

ax1.plot(df['Close'], color='r')
ax2.plot(df['rsi'], color = 'b')
ax2.plot(df['rsi_ema'], color = 'g')
# ax2.plot(x, y, color='b', linestyle='--')

In [225]:
#@title SHA

def SHA(df, length):
  import warnings
  warnings.filterwarnings("ignore")
  
  import matplotlib.pyplot as plt
  from bokeh.plotting import figure, output_file, show
  import bokeh
  from math import pi
  from bokeh.io import output_notebook
  from bokeh.models import Label
  from datetime import datetime as dt
  # Call once to configure Bokeh to display plots inline in the notebook.
  output_notebook()
  %matplotlib inline
  from math import pi
  from bokeh.models import Span
  w_l = length
  df['O'],df['H'],df['L'],df['C']= ta.ema(df['Open'], w_l),ta.ema(df['High'], w_l),ta.ema(df['Low'], w_l),ta.ema(df['Close'], w_l)
  df = df.dropna()
  df['C_1'] = (df['O']+df['H']+df['L']+df['O'])/4
  O,C = [df['O'][0]],[df['C'][0]]
  o,c = df['O'].shift(-1),df['C'].shift(-1)
  t= (o+c)/2
  r = t.shift(1)
  r[np.isnan(r)]=df['O'][0]
  df['O_1'] = t
  df['H_1'] = df[['H','O_1']].max(axis=1)
  df['L_1'] = df[['L','O_1']].min(axis=1)
  df['O_2'] = ta.ema(df['O_1'],w_l)
  df['L_2'] = ta.ema(df['L_1'],w_l)
  df['H_2'] = ta.ema(df['H_1'],w_l)
  df['C_2'] = ta.ema(df['C_1'],w_l)
  df['C_2'][-1] = df.C_1[-1]
  df = df.dropna()
# df = df[['O_2','H_2','L_2','C_2','Volume']]
  df = df.reset_index()
  return df[['Date','Open','High','Low','Close','O_2','H_2','L_2','C_2']]

In [161]:
#@title Consolidation
def is_consolidating(df, percentage = 2.5):
  recent_candlestick = df[-14:]
  # print(recent_candlestick)

  max_close = recent_candlestick['Close'].max()
  min_close = recent_candlestick['Close'].min()

  threshold = 1- (percentage/100)
  if min_close > (max_close*threshold):
    return True
  return False
  # print(f"The max close is {max_close} and min close is {min_close}")

def is_breaking_out(df, percentage = 2.5):
  last_close = df[-1:]['Close'].values[0]
  
  if is_consolidating(df[:-1], percentage = percentage):
    recent_closes = df[-22:-1]
    if last_close > recent_closes['Close'].max():
      return True
  return False

In [ ]:
#@title Initial design do not run this
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas_datareader as pdr
import pandas as pd
import numpy as np
import pandas_ta as ta
stock = 'nvda'
df = pdr.get_data_yahoo(f'{stock}',"12-1-2020")
t = SHA(df,3)
dfs = trima(df)
m = SHA(df,5)
candlestick = go.Candlestick(x=t['Date'],open=t['O_2'],high=t['C_2'],low=t['O_2'],close=t['C_2'], name = 'Trend Filter1')
candlestick1 = go.Candlestick(x=t['Date'],open=t['Open'],high=t['High'],low=t['Low'],close=t['Close'], name = 'Candlestick')
candlestick3 = go.Candlestick(x=m['Date'],open=m['O_2'],high=m['C_2'],low=m['O_2'],close=m['C_2'], name = 'Trend Filter2')
fig = go.Figure(data=[candlestick, candlestick1, candlestick3])
cs = fig.data[1]
cs1 = fig.data[0]
cs2 = fig.data[2]

fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)','paper_bgcolor': 'rgba(0, 0, 0, 0)'})
cs.increasing.fillcolor = '#FFFFFF';cs.increasing.line.color = '#000000';cs.decreasing.fillcolor = '#000000';cs.decreasing.line.color = '#000000'
cs.decreasing.line.width = 1;cs.increasing.line.width = 1
cs1.increasing.fillcolor = '#FF0000';cs1.increasing.line.color = '#FF0000';cs1.decreasing.fillcolor = '#0D600D';cs1.decreasing.line.color = '#0D600D'
cs1.decreasing.line.width = 1;cs1.increasing.line.width = 1
cs2.increasing.fillcolor = '#FF0000';cs2.increasing.line.color = '#FF0000';cs2.decreasing.fillcolor = '#0D600D';cs2.decreasing.line.color = '#0D600D'
cs2.decreasing.line.width = 1;cs2.increasing.line.width = 1

fig.add_traces(go.Scatter(x=dfs.index, y=dfs.HH,fill=None, mode='lines',line_color='green',line=dict(color='green',width=1),opacity=0.3, name = "Upper Resistance 1"))
fig.add_traces(go.Scatter(x=dfs.index, y=dfs.H,fill='tonexty',mode='lines', line_color='green',line=dict(color='green',width=1),opacity=0.3,name = "Upper Resistance 2"))   
fig.add_traces(go.Scatter(x=dfs.index, y=dfs.LL,fill=None, mode='lines',line_color='red',line=dict(color='red',width=1), name = "Lower Support 1"))
fig.add_traces(go.Scatter(x=dfs.index, y=dfs.L,fill='tonexty',mode='lines', line_color='red',line=dict(color='red',width=1), name = "Lower Support 2"))   

for line in b:
  fig.add_traces(go.Scatter(x = dfs.index, y = pd.Series(np.array([line for _ in range(len(dfs))])), mode = 'lines', line = dict(color = "black", width = 2),opacity=0.4, name = "VP S/R"))

fig.update_layout(title = f'{stock}', xaxis_title="Date", yaxis_title ='Price',xaxis_rangeslider_visible=False,showlegend = True, height=600)
fig.update_xaxes(showgrid=False,gridwidth=1, gridcolor='gray')
fig.update_yaxes(showgrid=False, gridwidth=1, gridcolor='gray')
fig.show()


In [162]:
!pip install finta
!pip install MarketProfile

In [266]:
stock = 'CAT'

In [267]:
#@title MArket Profile
from market_profile import MarketProfile

df = pdr.get_data_yahoo(f'{stock}',"12-1-2020")

class Analysis:
  
  def __init__(self, dataframe):
    self.dataframe = dataframe
    self.profile = []
    self.levels = []
    self.level = []

  # Diagram for candle stick 
  def candlestick(self):
    return self.dataframe

  # Calculation of Market Profile
  def market_profile(self):
    for i in range(1,len(self.dataframe),20):
      try:
        mp = MarketProfile(self.dataframe)
        mp_slice = mp[self.dataframe.index[0+i]: self.dataframe.index[20+i]]
        self.profile.append(mp_slice.poc_price)
      except:
        pass
    return self.profile


a = Analysis(df)


b = a.market_profile()

In [268]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from finta import TA
import numpy as np
#Load data
df = pdr.get_data_yahoo(f'{stock}',"12-1-2020")
v = df.copy() # for volatility
o = df.copy() # for OB/OS
efi = df.copy() # for EFI
t = SHA(df,3)
dfs = trima(df)
m = SHA(df,5)

# Volitility 

v['macd'] = TA.MACD(v,9,19,6)['MACD']
v['signal'] = TA.MACD(v,9,19,6)['SIGNAL']
# v['squeeze'] = ta.squeeze(v.High,v.Low, v.Close, 20,1.6,20,1, lazybear = True)['SQZ_ON']
v['s'] = TA.SQZMI(v)
v['squeeze'] = v.apply(lambda x: 1 if x['s'] == True else 0, axis = 1)
v['stc'] = (TA.STC(v,23,50,3,3))*(v['Close'].max()/100)
mid = (v['Close'].max()*.5)
upp = (v['Close'].max()*.75)
low = (v['Close'].max()*.25)

# OB/OS
o['WT1'] = TA.WTO(df)['WT1.'] 
o['WT2'] = TA.WTO(df)['WT2.']

# Mass Index  : Used to indentify trend exhaustion and plotted on same graph of OB/OS
# mass['MI'] = TA.MI(df)
# EFI
efi['EFI'] = TA.EFI(efi)
efi['color'] = np.where(efi['EFI']<0, 'red', 'green')

# Create subplots and mention plot grid size
fig = make_subplots(rows=4, cols=1, shared_xaxes=True, 
               vertical_spacing=0.05, 
               row_width=[0.2, 0.2,0.2,0.7]) # subplot_titles=(f'{stock}') if want to add title

# Candle stick chart
fig.add_trace(go.Candlestick(x=t['Date'],open=t['Open'],high=t['High'],low=t['Low'],close=t['Close'], 
                             name = 'Candlestick',increasing_line_color= '#000000', decreasing_line_color= '#000000',
                             decreasing_fillcolor = "#FFFFFF", increasing_fillcolor = "#000000", 
                             increasing_line_width=1,decreasing_line_width=1), row=1, col=1)

#Heikin ashi chart 1
fig.add_trace(go.Candlestick(x=t['Date'],open=t['O_2'],high=t['C_2'],low=t['O_2'],close=t['C_2'], 
                             name = 'Trend Filter1',increasing_line_color= '#FF0000', decreasing_line_color= '#0D600D',
                             decreasing_fillcolor = "#0D600D", increasing_fillcolor = "#FF0000", 
                             increasing_line_width=1,decreasing_line_width=1), row=1,col=1)

#Heikin ashi chart 2
fig.add_trace(go.Candlestick(x=m['Date'],open=m['O_2'],high=m['C_2'],low=m['O_2'],close=m['C_2'], 
                             name = 'Trend Filter2',increasing_line_color= '#FF0000', decreasing_line_color= '#0D600D',
                             decreasing_fillcolor = "#0D600D", increasing_fillcolor = "#FF0000", 
                             increasing_line_width=1,decreasing_line_width=1), row=1,col=1)

# SR Volume Profile
for line in b:
  fig.add_trace(go.Scatter(x = dfs.index, y = pd.Series(np.array([line for _ in range(len(dfs))])), mode = 'lines', line = dict(color = "red", width = 1), name = "VP S/R"))

# Upper and Lower Resistance
fig.add_trace(go.Scatter(x=dfs.index, y=dfs.HH,fill=None, mode='lines',line_color='green',line=dict(color='green',width=1),opacity=0.3, name = "Upper Resistance 1"),row=1,col=1)
fig.add_trace(go.Scatter(x=dfs.index, y=dfs.H,fill='tonexty',mode='lines', line_color='green',line=dict(color='green',width=1),opacity=0.3,name = "Upper Resistance 2"),row=1,col=1)   
fig.add_trace(go.Scatter(x=dfs.index, y=dfs.LL,fill=None, mode='lines',line_color='red',line=dict(color='red',width=1), name = "Lower Support 1"),row=1,col=1)
fig.add_trace(go.Scatter(x=dfs.index, y=dfs.L,fill='tonexty',mode='lines', line_color='red',line=dict(color='red',width=1), name = "Lower Support 2"),row=1,col=1)

# Squeeze and STC
fig.add_trace(go.Scatter(x=v.index, y=v.Close,mode='markers',marker_color=v['squeeze'], marker_size = 6, name = 'Volatility & Momentum'), row=2, col=1)
fig.add_trace(go.Scatter(x=v.index, y=v['stc'] ,fill='tozeroy', line_color='black',line=dict(color='black',width=1), opacity = 0.4, name = 'Momentum'), row=2, col=1)
fig.add_trace(go.Scatter(x = v.index, y = pd.Series(np.array([mid for _ in range(len(v))])),mode = 'lines',line = dict(color= 'black', width = 1), name = 'Midline'), row = 2, col=1)
fig.add_trace(go.Scatter(x = v.index, y = pd.Series(np.array([upp for _ in range(len(v))])),mode = 'lines',line = dict(color= 'red', width = 1,dash='dot'), name = 'Upper-line'), row = 2, col=1)
fig.add_trace(go.Scatter(x = v.index, y = pd.Series(np.array([low for _ in range(len(v))])),mode = 'lines',line = dict(color= 'green', width = 1,dash='dot'), name = 'Lower-line'), row = 2, col=1)

#Over Bought and Over Sold along with Mass Index
fig.add_trace(go.Scatter(x = o.index, y = o['WT1'],fill =None, mode = 'lines',line_color='green',line=dict(color='green',width=1),name='WT1'), row = 3, col = 1)
fig.add_trace(go.Scatter(x = o.index, y = o['WT2'], mode = 'lines',line_color='red',line=dict(color='red',width=1),name='WT2'), row = 3, col = 1)
fig.add_trace(go.Scatter(x = o.index, y = pd.Series(np.array([0 for _ in range(len(o))])), mode = 'lines', line = dict(color = 'black', width =1, dash = 'dot'), name = 'Zero Line'), row = 3,col = 1)

fig.add_trace(go.Bar(name='Bull and Bear',x=efi.index,y=efi.EFI,marker_color=efi['color']), row= 4, col = 1)



fig.update_layout(title = "Candlestick", yaxis_title ='Price',xaxis_rangeslider_visible=False,showlegend = False,barmode='stack', height=1000) #xaxis_title="Date"
fig.update_xaxes(showgrid=True,gridwidth=1, gridcolor='gray')
fig.update_yaxes(showgrid=False, gridwidth=1, gridcolor='gray')
fig.update_layout({'plot_bgcolor': 'rgba(0, 0, 0, 0)','paper_bgcolor': 'rgba(0, 0, 0, 0)'})

fig.show()

In [250]:
#@title
g = pdr.get_data_yahoo('HD',"12-1-2020")
g['stc'] = TA.STC(df,21,30)
g['signal'] = g.apply(lambda x: 1 if x['stc']>25 else -1 if x['stc']<75 else 0, axis = 1)
g['S'] = (g['signal']-g['signal'].shift(1))
# df['stc'].plot(figsize = (20,10))
# fig.add_trace(go.Scatter(x=dfs.index, y=dfs['stc'] ,fill='tozeroy'), row=2, col=1)

In [ ]:
#@title
x = []
y = []
p = []
for a in range(20,30,2):
  for t in range(50,60,2):
    g = pdr.get_data_yahoo('jpm',"1-1-2015")
    g['stc'] = TA.STC(g,a,b)
    g['signal'] = g.apply(lambda x: 1 if x['stc']>25 else -1 if x['stc']<75 else 0, axis = 1)
    g['S'] = (g['signal']-g['signal'].shift(1))
    initial_capital = 100000
    initial = 0
    buy = 0
    b = 0
    
    

    r = []
    for i in range(len(g)):
      if g['S'][i] == 2:
        buy_unit = initial_capital//g['Close'][i]
        buy += buy_unit
        initial = initial_capital - (buy_unit * g['Close'][i])
        initial_capital -= initial
        b += g['Close'][i]
        d = (g.index[i])
        # print(f"Stock was bought on {g.index[i]} at {g['Close'][i]}. Unit in stock {buy}. Remaining amount {initial}") 
      elif (g['S'][i] == -2) and (buy >0):
        sell = buy * g['Close'][i] 
        buy = 0
        net = initial + (sell* g['Close'][i])
        # print(f"Stock was sold on {g.index[i]} at {g['Close'][i]}. Return {np.round(g['Close'][i]/b,3)}")#. Time {g.index[i] - d}")
        r.append(g['Close'][i]/b)
        b= 0
        d= 0
    x.append(a)
    y.append(b)
    p.append((np.prod(r)-1)*100)
    print(f"The overall return is {(np.prod(r)-1)*100}% with final value {((np.prod(r))*initial_capital)-initial_capital}----> with parameter {a} {t}")

In [263]:
df = pdr.get_data_yahoo("mcd", "1-1-2020")
fib = TA.PIVOT_FIB(df)

In [265]:
[fib.groupby('pivot').resample('W', how = 'sum')]

In [262]:
fib

,pivot,s1,s2,s3,s4,r1,r2,r3,r4
Date,,,,,,,,,
2020-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-03,199.799998,198.657816,197.952175,196.809992,195.667810,200.942180,201.647821,202.790003,203.932186
2020-01-06,199.826670,199.177271,198.776072,198.126673,197.477275,200.476069,200.877268,201.526667,202.176066
2020-01-07,201.483337,200.176898,199.369779,198.063339,196.756900,202.789777,203.596896,204.903336,206.209775
2020-01-08,201.939997,201.111058,200.598938,199.769999,198.941060,202.768937,203.281056,204.109996,204.938935
...,...,...,...,...,...,...,...,...,...
2021-05-07,234.559998,233.612639,233.027360,232.080002,231.132643,235.507356,236.092635,237.039993,237.987352
2021-05-10,234.850001,234.189143,233.780864,233.120005,232.459147,235.510859,235.919138,236.579997,237.240855
2021-05-11,236.710002,235.632765,234.967247,233.890010,232.812773,237.787239,238.452757,239.529994,240.607231


In [3]:
#@title
df['trima'] = ta.trima(df['Close'],21)
df['range'] = ta.atr(df.High, df.Low, df.Close, 21) * 4
df['H'] = ta.wma((df['trima'] + df['range']),21)
df['L'] = ta.wma((df['trima'] - df['range']),21)
df['SD_UP'] = df['H'].rolling(20).std()
# df['SD_DOWN'] = ((df['trima'] - df['range'])).rolling(20).std()
df['HH'] = ta.wma((df['trima'] + df['range']),21)+(ta.atr(df.High, df.Low, df.Close,21))*1.5
df['LL'] = ta.wma((df['trima'] - df['range']),21)-(ta.atr(df.High, df.Low, df.Close,21))*1.5